Imports e Configuração

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Configuração para os gráficos ficarem bonitos
sns.set_theme(style="whitegrid")
%matplotlib inline

Carregar Dados Reais (do GitHub)

In [ ]:
# Lista das épocas que queremos carregar (URLs do repositório do Vaastav)
seasons = ["2021-22", "2022-23", "2023-24", "2024-25", "2025-26"]
base_url = "https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/"

all_data = []

print("🚀 A iniciar o download dos dados...")

for season in seasons:
    print(f"   -> A carregar época {season}...", end=" ")
    try:
        url = f"{base_url}{season}/gws/merged_gw.csv"
        
        # A CORREÇÃO ESTÁ AQUI: on_bad_lines='skip'
        # Isto diz ao pandas: "Se encontrares uma linha estragada, ignora e continua"
        df_season = pd.read_csv(url, on_bad_lines='skip', low_memory=False)
        
        # Adicionar identificador da época
        df_season['season'] = season
        
        all_data.append(df_season)
        print(f"✅ Sucesso! ({df_season.shape[0]} registos)")
        
    except Exception as e:
        print(f"❌ Erro grave na época {season}: {e}")

# Juntar tudo num único DataFrame
if all_data:
    df_granular = pd.concat(all_data, ignore_index=True)
    print(f"\n🎉 CONCLUÍDO! Total de registos para treino: {df_granular.shape[0]}")
else:
    print("\n⚠️ Nenhum dado foi carregado. Verifica a tua ligação.")

Preparar os Dados (Feature Selection)

In [ ]:
# 1. Escolher as colunas importantes
# 'value' = preço do jogador naquela semana
features = ['value', 'influence', 'creativity', 'threat', 'minutes', 'ict_index']
target = 'total_points'

# 2. Criar dataset limpo
# .copy() para evitar avisos do pandas
df_model = df_granular[features + [target]].copy()

# 3. Converter tudo para números (às vezes o CSV traz texto por engano)
for col in features:
    df_model[col] = pd.to_numeric(df_model[col], errors='coerce')

# 4. Remover linhas com erros (NaNs)
df_model = df_model.dropna()

# 5. Definir X e y
X = df_model[features]
y = df_model[target]

print("Amostra dos dados de treino (X):")
display(X.head())

Dividir em Treino e Teste

In [ ]:
# Dividir: 80% para treinar, 20% para testar
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Tamanho do Treino: {X_train.shape[0]} jogadores")
print(f"Tamanho do Teste: {X_test.shape[0]} jogadores")

Treinar o Modelo (Linear Regression)

In [ ]:
# 1. Criar o modelo
model = LinearRegression()

# 2. Treinar o modelo com os dados de treino
model.fit(X_train, y_train)

print("Modelo treinado com sucesso!")
print(f"Intercept (b): {model.intercept_:.2f}")
print("Coeficientes (Pesos W):")
for feature, coef in zip(features, model.coef_):
    print(f"  {feature}: {coef:.4f}")

Avaliar e Visualizar

In [ ]:
# Fazer previsões
y_pred = model.predict(X_test)

# Métricas
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print(f"📊 Resultados com {len(seasons)} épocas de histórico:")
print(f"   R2 Score: {r2:.4f} (Quanto mais perto de 1.0, melhor)")
print(f"   MSE: {mse:.2f}")

# Gráfico
plt.figure(figsize=(10, 6))
# Vamos usar apenas uma amostra de 500 pontos para o gráfico não ficar lento
plt.scatter(y_test[:500], y_pred[:500], alpha=0.4, color='purple', label='Jogos (Amostra)')
plt.plot([0, 25], [0, 25], color='red', linestyle='--', label='Previsão Perfeita')
plt.xlabel('Pontos Reais na Jornada')
plt.ylabel('Pontos Previstos pelo Modelo')
plt.title(f'Realidade vs. Previsão (Base de Dados: {df_granular.shape[0]} jogos)')
plt.legend()
plt.show()